In [ ]:
!git clone https://github.com/k-ganda/NJIA.git

Cloning into 'NJIA'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 36 (delta 5), reused 18 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 5.19 MiB | 7.22 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
# Change the current working directory to the 'NJIA' repository
%cd NJIA

# Verify your current working directory
!pwd

# List the contents of the repository to confirm you are in the right place
!ls

/content/NJIA
/content/NJIA
2_preprocessing  3_medASR_outputs  input_audio	new_colab_file.txt  README.md


In [ ]:
!pip install -q transformers accelerate torch


In [ ]:
import json
import re
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM


In [ ]:
!pip install -U llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 21.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4503265 sha256=6c83c85351f667b2687a2d7ab407714f6b6ba38f9a2757942b323772cf4e4285
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
MODEL_ID = "google/gemma-2b-it"

from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN') # Get HF token from Colab secrets

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype="auto",
    token=HF_TOKEN
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
with open("3_medASR_outputs/transcripts.json", "r") as f:
    transcripts = json.load(f)

len(transcripts)

4

In [ ]:
SYSTEM_PROMPT = """
You are a medical forensic documentation assistant.

Your task is to extract structured clinical facts from survivor testimony.
Do NOT summarize.
Do NOT add new information.
Do NOT assume intent.

Only extract facts explicitly stated or clearly implied.

Use neutral, clinical language.
"""

USER_PROMPT_TEMPLATE = """
Extract the following fields from the testimony below.

Fields:
- injury_type
- body_location
- injury_color_or_stage
- mechanism_of_injury
- timing_of_assault
- repeated_assault (yes/no/unclear)
- drug_facilitated_indicators
- survivor_uncertainty_notes

Return output as JSON only.

Testimony:
\"\"\"{transcript}\"\"\"
"""



In [ ]:
def extract_clinical_facts(transcript):
    prompt = SYSTEM_PROMPT + USER_PROMPT_TEMPLATE.format(transcript=transcript)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.0,
        do_sample=False
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract JSON safely
    json_match = re.search(r"\{.*\}", response, re.DOTALL)
    if json_match:
        return json.loads(json_match.group())
    else:
        return {"error": "Could not parse JSON"}


In [ ]:
structured_outputs = []

for item in tqdm(transcripts):
    extracted = extract_clinical_facts(item["transcript"])

    structured_outputs.append({
        "audio_id": item["audio_id"],
        "clinical_facts": extracted
    })



  0%|          | 0/4 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

100%|██████████| 4/4 [00:19<00:00,  4.84s/it]


In [ ]:
OUTPUT_PATH = "4_clinical_extractions.json"

with open(OUTPUT_PATH, "w") as f:
    json.dump(structured_outputs, f, indent=2)

OUTPUT_PATH


'4_clinical_extractions.json'

In [ ]:
import pprint

print("Example extracted clinical facts:\n")
pprint.pprint(structured_outputs[0])


Example extracted clinical facts:

{'audio_id': 'case4.wav',
 'clinical_facts': {'body_location': None,
                    'drug_facilitated_indicators': None,
                    'injury_color_or_stage': None,
                    'injury_type': None,
                    'mechanism_of_injury': 'Cord-like',
                    'repeated_assault': None,
                    'survivor_uncertainty_notes': 'These are red lines around '
                                                  'my arms.',
                    'timing_of_assault': None}}


In [ ]:
import os

OUTPUT_DIR = "." # Define OUTPUT_DIR as the current directory
readme_path = os.path.join(OUTPUT_DIR, "README.md")

# Example content for the README.md file
readme_content = """
## ## Clinical Text Postprocessing (MedGemma)

This stage transforms raw ASR transcripts into structured clinical and forensic facts using **MedGemma** from Google’s Health AI Developer Foundations (HAI-DEF).

### Objective
To bridge the gap between survivor speech and medical-legal documentation by extracting clinically relevant facts while preserving survivor intent and uncertainty.

---

### Model Usage
MedGemma is used for medical language understanding and clinical reasoning, enabling:
- Injury identification
- Mechanism-of-injury inference
- Timeline extraction
- Detection of drug-facilitated assault indicators
- Explicit capture of survivor uncertainty

This task requires domain-specific medical reasoning and cannot be reliably performed using general-purpose language models.

---

### Extraction Strategy
- Verbatim transcripts are passed to MedGemma
- A structured prompt enforces:
  - No summarization
  - No assumption
  - Neutral clinical language
- Output is constrained to JSON for downstream forensic formatting

---


"""

with open(readme_path, "w") as f:
    f.write(readme_content)

print(f"README.md created/updated at: {readme_path}")

README.md created/updated at: ./README.md
